In [ ]:
# Import libraries

# First, let’s import all of the modules, functions and objects we are going to use in this tutorial.

# Pandas for data handling
import pandas # https://pandas.pydata.org/
from pandas.plotting import scatter_matrix

# pretty tables
from IPython.display import display

# NumPy for numerical computing
import numpy # https://numpy.org/

# MatPlotLib + Seaborn for visualization
import matplotlib.pyplot as pl  # https://matplotlib.org/
import seaborn as sns   # https://seaborn.pydata.org/

# assessment
from sklearn import model_selection # for model comparisons
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
print('Loading data from file ...')  # Now let's load the data
dataset = pandas.read_csv('floats.csv') # default is header=infer, change if column names are not in first row
print('done \n')

print('Removing rows with missing data ...')  # Make things simple
dataset = dataset.dropna()  # default is to drop any row that contains at least one missing value
print('done \n')

In [ ]:
# Let's set up a problem: Can we predict 'callSign' using these three features: 'Depth', 'Temperature', 'Salinity' ?

print('Reading list of problem variables X and Y...')
X_name = [ 'Depth', 'Temperature', 'Salinity' ] # columns to focus on as predictors
X = dataset[X_name]   # only keep these columns as features
y_name = 'callSign'     # column to focus on as target
y = dataset[y_name]   # only keep this column as label 
print('done \n')

In [ ]:
# Split-out the test dataset for summative evaluation

# We reset the random number seed before each run to ensure that the evaluation of each algorithm is performed using exactly the same data splits. It ensures the results are directly comparable.

# Train, test split
print('Partitioning data into two parts: train (for formative development) and test (for summative evaluation) ...')
test_size = 0.20   # means 20 percent
seed = 42 # setting the seed allows for repeatability

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size, random_state=seed)
print('done \n')

In [ ]:
# Chose the formative scoring method

print('Reading list of scoring methods to use during model development ...')
scoring = 'accuracy'
print('done \n')

In [ ]:
# Chose the Algorithms

print('Reading list of algorithms to train ...')
models = []
models.append(('LR', LogisticRegression(multi_class='multinomial', class_weight=None, solver='lbfgs', max_iter=250)))
models.append(('Ridge', RidgeClassifier(alpha=10)))
models.append(('kNN', KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', p=2)))
print('done \n')

In [ ]:
# Now it is time to develop (train and validate) the models on the formative data set

k4folds = 5   # This will split our formative dataset into five parts;
results = []
names = []
for name, model in models:   # Select each model in turn
    print(" ++ NOW WORKING ON ALGORITHM %s ++" % name)
    print("Splitting data into %s folds" % k4folds)
    kfold = model_selection.KFold(n_splits=k4folds, random_state=seed, shuffle=True)   # fit the model using four parts at a time and then validate it on the oher part that was set aside; and repeat five times.
    print("Training model and validating it on each fold")
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "algorithm %s %s results: mean = %f (std = %f)" % (name, scoring, cv_results.mean(), cv_results.std())
    print(msg)
    
print('done \n')   # Note that we have not used any of the test data yet!

In [ ]:
# Graphical comparison 
# We can also create a plot of the model evaluation results 
# and compare the spread and the mean accuracy of each model. 
# There is a population of accuracy measures for each algorithm 
# because each algorithm was evaluated several times (k fold cross validation).
fig = pl.figure()
fig.suptitle('Algorithm Comparison based on %s' % scoring)
ax = fig.add_subplot(111)
pl.boxplot(results)
ax.set_xticklabels(names)
pl.show()

In [ ]:
# First we retrain the most capable model using ALL of the training data
selected_model = KNeighborsClassifier()
selected_model.fit(X_train, y_train)

# Next we use the trained KNN model on the test set and summarize the results 
# as a final accuracy score, a confusion matrix and a classification report.
predictions = selected_model.predict(X_test)
print(" ++++ Detailed classification report for the selected model ++++ " )
print("Algorithm %s " % selected_model)
print("This model was trained and tuned on the development set using CV.")
print("The following results are computed on the separate test set:")
#
predictions = selected_model.predict(X_test)
cm = confusion_matrix(y_test, predictions)
df_cm = pandas.DataFrame(cm, columns=numpy.unique(y_test), index = numpy.unique(y_test))
sns.heatmap(df_cm, square=True, annot=True, fmt='d', cbar=False )
pl.xlabel('predicted')
pl.ylabel('true')
pl.show()
#
print('\n clasification report:\n', classification_report(y_test, predictions))
print('\n')        
print('done \n')

In [ ]:
# PS: We can do a similar process to chose between different hyperparameters for a model

# Train and validate all the different model versions using the (formative) data set
hyperparameters = {'n_neighbors':[3, 4, 5, 6, 7], 'weights':['uniform','distance']}
clf = GridSearchCV(selected_model, hyperparameters, cv=5, scoring=scoring)
clf.fit(X_train, y_train)

# Select the Most Promising Model version
print("Best hyperparameters set found on development (formative) data set:")
print(clf.best_params_)
print("Grid scores on development (formative) data set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print('\n')

# Report the independent final verdict using the summative data set
print("Detailed classification report:")
print("The model was trained and tuned using cross-validation on the full development (formative) data set.")
print("The reported summative evaluation scores were computed on a separate test data set.")
print('\n')
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print('done \n')